<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Advanced_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [ ]:
!pip install -q llama-index==0.10.49 llama-index-vector-stores-chroma==0.1.9 llama-index-llms-gemini==0.1.11 google-generativeai==0.5.4 openai==1.35.3 chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 603.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

## Language Model and Embedding Model

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

# Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")

Settings.llm = Gemini(model="models/gemini-1.5-flash", temperature=1, max_tokens=512)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

**Note: You can create a vector store from scratch using the code below, or you can load it from Hugging Face using the code provided in this notebook.**

# Create a Vector Store


In [ ]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = chroma_client.create_collection("ai_tutor_knowledge")

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.storage_context import StorageContext

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load the Dataset (JSON)


## Download


In [ ]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

ai_tutor_knowledge.jsonl:   0%|          | 0.00/6.96M [00:00<?, ?B/s]

# Read File


In [ ]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]

len(ai_tutor_knowledge)

762

# Convert to Document obj


In [ ]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)

# Transforming


In [ ]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    include_metadata=True,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [ ]:
documents = [i for i in doc if i.metadata['tokens']<8000]
nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
len(nodes)

22083

In [ ]:
nodes[0]

TextNode(id_='8f7d6a43-800f-4f81-a199-8fbca18d66fe', embedding=None, metadata={'window': 'Github Repo : https://github.com/vaibhawkhemka/ML-Umbrella/tree/main/MLops/Model_Deployment/Bert_Kubernetes_deployment   Model development is useless if you dont deploy it to production  which comes with a lot of issues of scalability and portability.    I have deployed a basic BERT model from the huggingface transformer on Kubernetes with the help of docker  which will give a feel of how to deploy and manage pods on production.    Model Serving and Deployment:ML Pipeline:Workflow:   Model server (using FastAPI  uvicorn) for BERT uncased model    Containerize model and inference scripts to create a docker image    Kubernetes deployment for these model servers (for scalability)  Testing   Components:Model serverUsed BERT uncased model from hugging face for prediction of next word [MASK].  Inference is done using transformer-cli which uses fastapi and uvicorn to serve the model endpoints   Server st

## Index creation

In [ ]:
from llama_index.core import VectorStoreIndex

# Add the documents to the database and create Index / embeddings
index = VectorStoreIndex(nodes, storage_context=storage_context,show_progress =True)

In [ ]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore-windowed.zip ai_tutor_knowledge

  adding: ai_tutor_knowledge/ (stored 0%)
  adding: ai_tutor_knowledge/3678b80c-fc61-407c-b97c-a100ba9d8cf6/ (stored 0%)
  adding: ai_tutor_knowledge/3678b80c-fc61-407c-b97c-a100ba9d8cf6/header.bin (deflated 56%)
  adding: ai_tutor_knowledge/3678b80c-fc61-407c-b97c-a100ba9d8cf6/data_level0.bin (deflated 17%)
  adding: ai_tutor_knowledge/3678b80c-fc61-407c-b97c-a100ba9d8cf6/link_lists.bin (deflated 78%)
  adding: ai_tutor_knowledge/3678b80c-fc61-407c-b97c-a100ba9d8cf6/length.bin (deflated 67%)
  adding: ai_tutor_knowledge/3678b80c-fc61-407c-b97c-a100ba9d8cf6/index_metadata.pickle (deflated 44%)
  adding: ai_tutor_knowledge/chroma.sqlite3 (deflated 73%)


# Load Indexes


**Note: If you created the vector store from scratch, please uncomment out the three code blocks/cells below.**

In [ ]:
# from huggingface_hub import hf_hub_download
# vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore-windowed.zip",repo_type="dataset",local_dir="/content")

In [ ]:
# !unzip vectorstore-windowed.zip

In [ ]:
# import chromadb
# from llama_index.vector_stores.chroma import ChromaVectorStore
# from llama_index.core import VectorStoreIndex

# # Create your index
# db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
# chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# # Create your index
# vector_index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = index.as_query_engine(
    llm=Settings.llm,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [ ]:
response = query_engine.query("Write about naive RAG and Speculative RAG?")
print(response)

Naive RAG uses a single large language model to both retrieve information from external sources and generate responses. Speculative RAG, on the other hand, employs a specialized, smaller language model to generate multiple drafts from different subsets of retrieved documents, creating diverse perspectives.  These drafts are then verified by a larger, more generalist language model, enhancing comprehension and mitigating potential bias from relying on a single set of retrieved documents. 



In [ ]:
for idx, item in enumerate(response.source_nodes):
    print("Source ", idx + 1)
    print("Original Text:", item.node.metadata["original_text"])
    print("Window:", item.node.metadata["window"])
    print("----")

Source  1
Original Text: In this work, we introduce SPECULATIVE RAG – a framework that leverages a larger generalist LM to efficiently verify multiple RAG drafts produced in parallel by a smaller, distilled specialist LM. 
Window: **Abstract**Retrieval augmented generation (RAG) combines the generative abilities of large language models (LLMs) with external knowledge sources to provide more accurate and up-to-date responses.  Recent RAG advancements focus on improving retrieval outcomes through iterative LLM refinement or self-critique capabilities acquired through additional instruction tuning of LLMs.  In this work, we introduce SPECULATIVE RAG – a framework that leverages a larger generalist LM to efficiently verify multiple RAG drafts produced in parallel by a smaller, distilled specialist LM.  Each draft is generated from a distinct subset of retrieved documents, offering diverse perspectives on the evidence while reducing input token counts per draft.  This approach enhances comp